In [2]:
from google.colab import files
uploaded = files.upload()

Saving company_esg_financial_dataset.csv to company_esg_financial_dataset.csv


In [4]:
import pandas as pd

df = pd.read_csv('company_esg_financial_dataset.csv')
df.head()

,CompanyID,CompanyName,Industry,Region,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,CarbonEmissions,WaterUsage,EnergyConsumption
0,1,Company_1,Retail,Latin America,2015,459.2,6.0,337.5,NaN,57.0,60.7,33.5,76.8,35577.4,17788.7,71154.7
1,1,Company_1,Retail,Latin America,2016,473.8,4.6,366.6,3.2,56.7,58.9,32.8,78.5,37314.7,18657.4,74629.4
2,1,Company_1,Retail,Latin America,2017,564.9,5.2,313.4,19.2,56.5,57.6,34.0,77.8,45006.4,22503.2,90012.9
3,1,Company_1,Retail,Latin America,2018,558.4,4.3,283.0,-1.1,58.0,62.3,33.4,78.3,42650.1,21325.1,85300.2
4,1,Company_1,Retail,Latin America,2019,554.5,4.9,538.1,-0.7,56.6,63.7,30.0,76.1,41799.4,20899.7,83598.8


# Task
Preprocess the dataset "data.csv" by handling missing values, encoding categorical features, scaling numerical features, and performing feature engineering. Display the preprocessed data and summarize the steps taken.

## Handle missing values

### Subtask:
Identify and handle missing values in the dataset, potentially by imputation or removal.


**Reasoning**:
Identify missing values and their counts per column to understand the extent of missingness and decide on the imputation strategy.



In [5]:
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
CompanyID               0
CompanyName             0
Industry                0
Region                  0
Year                    0
Revenue                 0
ProfitMargin            0
MarketCap               0
GrowthRate           1000
ESG_Overall             0
ESG_Environmental       0
ESG_Social              0
ESG_Governance          0
CarbonEmissions         0
WaterUsage              0
EnergyConsumption       0
dtype: int64


**Reasoning**:
Based on the identified missing values, the 'GrowthRate' column has 1000 missing values. Since 'GrowthRate' is a numerical column, imputing with the mean is a reasonable strategy to handle these missing values without losing a significant amount of data.



In [6]:
df['GrowthRate'].fillna(df['GrowthRate'].mean(), inplace=True)
print("Missing values after imputation:")
print(df.isnull().sum())

Missing values after imputation:
CompanyID            0
CompanyName          0
Industry             0
Region               0
Year                 0
Revenue              0
ProfitMargin         0
MarketCap            0
GrowthRate           0
ESG_Overall          0
ESG_Environmental    0
ESG_Social           0
ESG_Governance       0
CarbonEmissions      0
WaterUsage           0
EnergyConsumption    0
dtype: int64


/tmp/ipython-input-6-1228591078.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['GrowthRate'].fillna(df['GrowthRate'].mean(), inplace=True)


## Handle categorical features

### Subtask:
Identify and encode categorical features using techniques like one-hot encoding.


**Reasoning**:
Identify categorical columns and apply one-hot encoding to them.



In [7]:
categorical_cols = df.select_dtypes(include=['object']).columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
display(df_encoded.head())

,CompanyID,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,...,Industry_Retail,Industry_Technology,Industry_Transportation,Industry_Utilities,Region_Asia,Region_Europe,Region_Latin America,Region_Middle East,Region_North America,Region_Oceania
0,1,2015,459.2,6.0,337.5,4.83037,57.0,60.7,33.5,76.8,...,True,False,False,False,False,False,True,False,False,False
1,1,2016,473.8,4.6,366.6,3.20000,56.7,58.9,32.8,78.5,...,True,False,False,False,False,False,True,False,False,False
2,1,2017,564.9,5.2,313.4,19.20000,56.5,57.6,34.0,77.8,...,True,False,False,False,False,False,True,False,False,False
3,1,2018,558.4,4.3,283.0,-1.10000,58.0,62.3,33.4,78.3,...,True,False,False,False,False,False,True,False,False,False
4,1,2019,554.5,4.9,538.1,-0.70000,56.6,63.7,30.0,76.1,...,True,False,False,False,False,False,True,False,False,False


## Scale numerical features

### Subtask:
Scale numerical features to a similar range to improve model performance.


**Reasoning**:
Identify the numerical columns to scale, excluding identifiers and encoded boolean columns, then apply StandardScaler to these columns and update the dataframe.



In [8]:
numerical_cols_to_scale = df_encoded.select_dtypes(include=['number']).columns.tolist()
exclude_cols = ['CompanyID', 'Year'] + [col for col in numerical_cols_to_scale if df_encoded[col].dtype == bool]
numerical_cols_to_scale = [col for col in numerical_cols_to_scale if col not in exclude_cols]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_encoded[numerical_cols_to_scale] = scaler.fit_transform(df_encoded[numerical_cols_to_scale])

display(df_encoded.head())

,CompanyID,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,...,Industry_Retail,Industry_Technology,Industry_Transportation,Industry_Utilities,Region_Asia,Region_Europe,Region_Latin America,Region_Middle East,Region_North America,Region_Oceania
0,1,2015,-0.422453,-0.559520,-0.326723,-9.884318e-17,0.150047,0.160017,-0.948854,0.988553,...,True,False,False,False,False,False,True,False,False,False
1,1,2016,-0.420989,-0.719369,-0.325994,-1.814398e-01,0.131171,0.092767,-0.978827,1.055687,...,True,False,False,False,False,False,True,False,False,False
2,1,2017,-0.411851,-0.650862,-0.327327,1.599161e+00,0.118587,0.044198,-0.927446,1.028044,...,True,False,False,False,False,False,True,False,False,False
3,1,2018,-0.412503,-0.753622,-0.328088,-6.599762e-01,0.212967,0.219794,-0.953136,1.047789,...,True,False,False,False,False,False,True,False,False,False
4,1,2019,-0.412894,-0.685115,-0.321698,-6.154612e-01,0.124879,0.272099,-1.098715,0.960909,...,True,False,False,False,False,False,True,False,False,False


## Feature engineering

### Subtask:
Create new features from existing ones if necessary.


**Reasoning**:
Create the new features as per the instructions and display the updated dataframe.



In [9]:
df_encoded['ESG_Score_Interaction'] = df_encoded['ESG_Overall'] * df_encoded['ESG_Environmental'] * df_encoded['ESG_Social'] * df_encoded['ESG_Governance']

df_encoded['Financial_Health'] = df_encoded['ProfitMargin'] / df_encoded['Revenue']
df_encoded['Financial_Health'].replace([float('inf'), float('-inf')], pd.NA, inplace=True)
mean_financial_health = df_encoded['Financial_Health'].mean()
df_encoded['Financial_Health'].fillna(mean_financial_health, inplace=True)

display(df_encoded.head())

/tmp/ipython-input-9-1581690971.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_encoded['Financial_Health'].replace([float('inf'), float('-inf')], pd.NA, inplace=True)
/tmp/ipython-input-9-1581690971.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing

,CompanyID,Year,Revenue,ProfitMargin,MarketCap,GrowthRate,ESG_Overall,ESG_Environmental,ESG_Social,ESG_Governance,...,Industry_Transportation,Industry_Utilities,Region_Asia,Region_Europe,Region_Latin America,Region_Middle East,Region_North America,Region_Oceania,ESG_Score_Interaction,Financial_Health
0,1,2015,-0.422453,-0.559520,-0.326723,-9.884318e-17,0.150047,0.160017,-0.948854,0.988553,...,False,False,False,False,True,False,False,False,-0.022521,1.324454
1,1,2016,-0.420989,-0.719369,-0.325994,-1.814398e-01,0.131171,0.092767,-0.978827,1.055687,...,False,False,False,False,True,False,False,False,-0.012574,1.708758
2,1,2017,-0.411851,-0.650862,-0.327327,1.599161e+00,0.118587,0.044198,-0.927446,1.028044,...,False,False,False,False,True,False,False,False,-0.004997,1.580334
3,1,2018,-0.412503,-0.753622,-0.328088,-6.599762e-01,0.212967,0.219794,-0.953136,1.047789,...,False,False,False,False,True,False,False,False,-0.046747,1.826948
4,1,2019,-0.412894,-0.685115,-0.321698,-6.154612e-01,0.124879,0.272099,-1.098715,0.960909,...,False,False,False,False,True,False,False,False,-0.035874,1.659300
